In [26]:
import polars as pl
import datetime as dt

In [38]:
(
    pl.read_parquet('./municipio_populacao.parquet')
    .join(
        other=pl.read_excel('../territorio/municipio_area.xlsx'),
        on='municipio_codigo_7',
        how='left'
    )
    .join(
        other=pl.read_excel('../territorio/uf_regiao.xlsx'),
        on='uf_codigo',
        how='left'
    )
    .join(
        other=pl.read_excel('../territorio/uf_codigo.xlsx'),
        on='uf_codigo',
        how='left'
    )
    .join(
        other=pl.read_parquet('../territorio/municipio_completo.parquet').select(pl.col(['municipio_codigo_7','municipio_nome'])),
        on='municipio_codigo_7',
        how='left'
    )
    .with_columns(
        pl.col('populacao')
        .truediv(pl.col('area'))
        .alias('densidade')
    )
    .select(
        'aaaa',
        'regiao',
        'uf_nome',
        'municipio_nome',
        'populacao',
        'area',
        'densidade'
    )
    .rename({
        'aaaa':'ano',
        'uf_nome':'estado',
        'municipio_nome':'municipio',
        'regiao':'Região',
    })
    .filter(
        pl.col('ano') == dt.datetime(2024,1,1)
    )
    # .drop_nulls()
    .sort(['area'], descending=True)
).write_csv('./p-densidade-municipio.csv')